In [1]:
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup as BS
import requests as rq


In [2]:
base_url_1 = 'https://www.property24.com/apartments-for-sale/'
base_url_2 = '/johannesburg/gauteng/5846'
n = 1
pager = 'p'+str(n)
areas = ['rosebank','norwood','parktown-north','illovo','hyde-park','parkwood','killarney','craighall-park','saxonworld','houghton-estate']
code = ['5846','4363','4391',]

In [3]:
page = rq.get('https://www.property24.com/apartments-for-sale/illovo/sandton/gauteng/5833?sp=so%3dPriceLow#SortOrder')

In [4]:
all_areas_page_iter_1 = 'https://www.property24.com/apartments-for-sale/advanced-search/results/p'
all_areas_page_iter_2= '?sp=s%3d5833%2c4388%2c5846%2c5832%2c5926%2c4390%2c5821%2c5779%2c5837%2c4391%2c4363%2c12133%26pt%3d3000000%26bd%3d2'

In [266]:
# all apartments 364
flats_url_1 = 'https://www.property24.com/apartments-for-sale/advanced-search/results/p'
flats_url_2= '?sp=cid%3d8%2c109%2c5%2c13%2c102%26s%3d5833%2c4388%2c5846%2c5832%2c5926%2c4390%2c5821%2c5779%2c5837%2c4391%2c4363%2c12133%2c5779%2c4404%2c4544%2c4386%2c11700%2c4414%2c5783%2c4258%2c4406%2c5176%2c5811%2c5857%2c5825%2c4145%2c4545%2c1574%2c1593%2c4217%2c3869%2c4327%2c4258%2c5847%2c5860%2c4455%2c4288%2c4251%2c695%2c13857%2c5290%2c1737%2c4681'

In [ ]:
# Flats + Houses
flats_houses_url = 'https://www.property24.com/for-sale/advanced-search/results/p'
flats_houses_url = '?sp=cid%3d8%2c109%2c5%2c13%2c102%26s%3d5833%2c4388%2c5846%2c5832%2c5926%2c4390%2c5821%2c5779%2c5837%2c4391%2c4363%2c12133%2c4404%2c4544%2c4386%2c11700%2c4414%2c5783%2c4258%2c4406%2c5176%2c5811%2c5857%2c5825%2c4145%2c4545%2c1574%2c1593%2c4217%2c3869%2c4327%2c5847%2c5860%2c4455%2c4288%2c4251%2c695%2c13857%2c5290%2c1737%2c4681&PropertyCategory=House%2cApartmentOrFlat%2cTownhouse'

In [247]:
all_areas_page = 'https://www.property24.com/apartments-for-sale/advanced-search/results?sp=s%3d5833%2c4388%2c5846%2c5832%2c5926%2c4390%2c5821%2c5779%2c5837%2c4391%2c4363%2c12133%26pt%3d3000000%26bd%3d2'
def find_list_in_page(url):
    page = rq.get(url)
    soup = BS(page.content,'html.parser')
    listing_cont = soup.find_all('div',{'class':'js_listingResultsContainer'})
    listings = listing_cont[-1].find_all("div", {"data-listing-number": True})
    return listings
listings = find_list_in_page(all_areas_page)

In [279]:
def get_property_info(listings):
    links = []
    images = []
    descs = []
    titles = []
    areas = []
    addrs = []
    prices = []
    beds = []
    baths = []
    squares = []
    parkings = []
    for l in listings:
#         print(l['data-listing-number'])
        if 'href' in l.find('a').attrs:
            links.append(l.find('a')['href'])
        elif 'data-href' in l.find('a').attrs:
            links.append(l.find('a')['data-href'])
        else:
            links.append('')
        if l.find('span',{'class':'p24_price'}):
            prices.append(str(l.find('span',{'class':'p24_price'}).text).strip())
        elif l.find('div',{'class':'p24_price'}):
            prices.append(str(l.find('div',{'class':'p24_price'}).text).strip())
        areas.append(l.find('span',{'class':'p24_location'}).text)
        if l.find('span',{'class':'p24_address'}) :
            addrs.append(l.find('span',{'class':'p24_address'}).text)
        else : 
            addrs.append('')
            
        if l.find('span',{'class':'p24_icons'}):
            icons = l.find('span',{'class':'p24_icons'})
        elif  l.find('div',{'class':'p24_icons'}):
            icons = l.find('div',{'class':'p24_icons'})
            
        if icons.find('span',{'title':'Bedrooms'}):
            beds.append(str(icons.find('span',{'title':'Bedrooms'}).text).strip())
        else:
            beds.append(0)
        if icons.find('span',{'title':'Bathrooms'}):
            baths.append(str(icons.find('span',{'title':'Bathrooms'}).text).strip())
        else:
            baths.append(0)
        if icons.find('span',{'title':'Parking Spaces'}):
            parkings.append(str(icons.find('span',{'title':'Parking Spaces'}).text).strip())
        else:
            parkings.append(0)
        if icons.find('span',{'title':'Floor Size'}):
            squares.append(str(icons.find('span',{'title':'Floor Size'}).text).strip())
        elif icons.find('span',{'title':'Erf Size'}):
            squares.append(str(icons.find('span',{'title':'Erf Size'}).text).strip())
        else:
            squares.append(0)
            
            
        
        if l.find('span',{'class':'js_listingTileImageHolder p24_image'}):
            ll = l.find('span',{'class':'js_listingTileImageHolder p24_image'}).find_next()
            if 'src' in ll.attrs :
                images.append(ll['src'])
                descs.append(ll['alt'])        
                titles.append(ll['title'])
            else :
                images.append('')
                descs.append('')
                titles.append('')
        elif l.find('div',{'class':'p24_promotedImage'}):
            ll = l.find('div',{'class':'p24_promotedImage'}).find_next()
            if 'src' in ll.attrs :
                images.append(ll['src'])
                descs.append(ll['alt'])        
                titles.append(ll['title'])
            else :
                images.append('')
                descs.append('')
                titles.append('')

        
    df = pd.DataFrame(columns=['links','images','descs','titles','areas','addrs','prices','beds','baths','squares','parkings'])
    df['links'] = links
    df['images'] = images
    df['descs'] = descs
    df['titles'] = titles
    df['areas'] = areas
    df['addrs'] = addrs
    df['prices'] = prices
    df['beds'] = beds
    df['baths'] = baths
    df['squares'] = squares
    df['parkings'] = parkings
    return df

In [278]:

test_url = flats_url_1 + str('102') + flats_url_2
listings = find_list_in_page(test_url)
x = get_property_info(listings[1:])
# print(test.find('span',{'title':'Bedrooms'}).text)
# print(test.find('span',{'title':'Bathrooms'}).text)
# print(test.find('span',{'title':'Parking Spaces'}).text)
# print(test.find('span',{'title':'Floor Size'}).text)
# for i in range(38,42):
#     print(i)    
#     print(listings[i]['data-listing-number'], '\n\n')
#     icons = listings[i].find('span',{'class':'p24_icons'})
#     print(icons)
#     print('###############################################################################',icons.find('span',{'title':'Floor Size'}),'###############################################################################')
#     print(icons.find('span',{'title':'Floor Size'}).text)
    


108454683
108308203
108308203
108353744
108353744
108361915
108361915
108554988
108554988
108584132
108584132
108585073
108585073
108598220
108598220
108490445
108490445
108549183
108549183
108477278
108477278
108493138
108493138
108304858
108304858
108558168
108558168
108481624
108481624
108433044
108433044
108298582
108298582
108592986
108592986
108291021
108291021
108290445
108290445
108414212
108414212


In [260]:
df_test = get_property_info(listings[1:])

In [261]:
df_test.head()

,links,images,descs,titles,areas,addrs,prices,beds,baths,squares,parkings
0,/for-sale/killarney/johannesburg/gauteng/4388/...,https://images.prop24.com/233290227/Crop318x212,A truly stunning apartment at an excellent pri...,2 Bedroom Apartment / flat for sale in Killarn...,Killarney,26 1st Street,R 2 700 000,2,1.5,184 m²,2
1,/for-sale/killarney/johannesburg/gauteng/4388/...,https://images.prop24.com/233290227/Crop318x212,A truly stunning apartment at an excellent pri...,2 Bedroom Apartment / flat for sale in Killarn...,Killarney,26 1st Street,R 2 700 000,2,1.5,184 m²,2
2,/for-sale/craighall-park/johannesburg/gauteng/...,https://images.prop24.com/233340205/Crop318x212,This delightful north facing home is spacious ...,2 Bedroom Apartment / flat for sale in Craigha...,Craighall Park,28 Clarence Avenue,R 1 999 500,2,2,128 m²,2
3,/for-sale/craighall-park/johannesburg/gauteng/...,https://images.prop24.com/233340205/Crop318x212,This delightful north facing home is spacious ...,2 Bedroom Apartment / flat for sale in Craigha...,Craighall Park,28 Clarence Avenue,R 1 999 500,2,2,128 m²,2
4,/for-sale/rosebank/johannesburg/gauteng/5846/1...,/blank.gif,2 Bedroom 2 Bathroom Apartment in The Tyrwhitt...,2 Bedroom Apartment / flat for sale in Roseban...,Rosebank,20 Tyrwhitt Avenue,R 2 780 000,2,2,80 m²,0


In [ ]:
def run_through(num_pages, link_1,link_2):
    dfs = []
    for num in range(1,num_pages):
        print(num)
        new_link = link_1 + str(num) + link_2
        listings = find_list_in_page(new_link)
        dfs.append(get_property_info(listings[1:]))
    return dfs
dfs = run_through(364,flats_url_1,flats_url_2)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


In [ ]:
final_df = pd.concat(dfs)
final_df.to_excel('property24_data_flats_jhb.xlsx')

In [ ]:
final_df.head()

In [ ]:
final_df.info()